<a href="https://colab.research.google.com/github/mathaushuber/MSc-Computer-Science/blob/main/8_puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
from copy import deepcopy
import numpy as np
from itertools import combinations
from IPython.display import Image
import pprint

In [3]:
Image(url='https://lh7-us.googleusercontent.com/EMbfMxUTGze7-3Nn5YrVqosIO041Le7DOS4qAiCrDPcSyLqhjIq0FjlRvuxlyNhGTvvtpmDnF6SY-yPEjfCZlsKREXabRw3AUWJaRMk6joiqYwRBDlitlLvtP5aE0wfrXL-dl9X7xW3uaFkWr5S2ji8',width=300)

In [4]:
class Estado:
    def __init__(self, array):
        self.array = np.array(array)

    def __repr__(self):
        return str(self.array[0]) + '\n' + str(self.array[1]) + '\n' + str(self.array[2]) + '\n'

    def __eq__(self, other):
        return (self.array == other.array).all()


class FilaPrioridade:
    def __init__(self):
        self.list = []

    def put(self, item, priority):
        self.list.append((priority, item))
        self.list.sort(key=lambda x: x[0])

    def get(self):
        return self.list.pop(0)[1]

    def EspacoVazio(self):
        return len(self.list) == 0

class No:
    def __init__(self, estadoInicial):
        self.estado = estadoInicial
        self.pos = self.getPos()
        self.caminho = [estadoInicial]

    def AlcancaNovoEstado(self, novoEstado):
        novoNo = deepcopy(self)
        novoNo.estado = novoEstado
        novoNo.pos = novoNo.getPos()
        novoNo.caminho.append(novoEstado)

        return novoNo

    def getPos(self):
        for i in range(3):
            for j in range(3):
                if self.estado.array[i][j] == 0:
                    return i, j

def VerificaPosicao(no, estadoParada):
    return no.estado == estadoParada

In [5]:
posicaoFinal = Estado([[1,2,3], [4,5,6], [7,8,0]])
posicaoInicial = Estado([[5, 2, 6], [1, 0, 3], [4, 7, 8]])
print("Posição inicial:\n" + str(posicaoInicial))
print("Posição final:\n" + str(posicaoFinal))

Posição inicial:
[5 2 6]
[1 0 3]
[4 7 8]

Posição final:
[1 2 3]
[4 5 6]
[7 8 0]




O contador é incrementado a cada iteração. Se a fronteira estiver vazia, a função retorna False, indicando que não há solução. O nó na frente da fila é removido e depois processado. Se a posição atual do nó for a posição final, a função imprime o número de movimentos e retorna o caminho até a solução.

Para cada possível movimento (cima, baixo, esquerda, direita), é verificado se o movimento é válido. Movimentos são considerados válidos se resultam em posições dentro dos limites do tabuleiro (3x3). Se o movimento é válido, um novo estado é gerado copiando o estado atual e trocando a posição do zero com a nova posição. Um novo nó é criado a partir do novo estado. Se o novo estado não está no caminho atual (evitando ciclos), ele é adicionado à fronteira.

A busca em largura explora todos os nós no nível atual antes de explorar os nós no próximo nível. Funciona como uma fila, onde os nós são adicionados ao final da fila e removidos do início.

**Vantagens:**


*   Completa: Sempre encontra uma solução se uma solução existir.
*   Ótima: Encontra a solução com o menor número de movimentos.


**Desvantagens:**


*   Consumo de Memória: Pode consumir muita memória, especialmente para problemas grandes, pois precisa armazenar todos os nós no nível atual antes de passar para o próximo.
*   Lentidão: Pode ser lenta se a solução estiver longe do nó inicial, pois explora todos os nós em cada nível antes de ir para o próximo nível.



In [21]:
def BuscaLargura(estadoInicial, estadoParada):
    no = No(estadoInicial)
    frg = [no]
    cnt = 0
    while True:
        cnt += 1

        if frg == []:
            return False
        no = frg.pop(0)
        #Se a posição do quebra cabeça for a posição desejada paramos o laço
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(len(no.caminho) -1))
            return no.caminho

        x, y = no.pos
        #A lógica da implementação para validação de um movimento é baseada em um laço
        #duplo de repetição. A primeira variável i representa a linha e j representa as colunas para aquela
        #linha. A validação do movimento é feita através da identificação da posição do número 0.
        #Uma vez encontrado adiciona-se e subtrai-se 1 no valor da linha para testar se é possível
        #movimentar para cima e para baixo. O mesmo teste é realizado para a coluna, porém nesse
        #caso é testado a possibilidade de mover-se para direita e esquerda. O movimento é valido se o
        #valor adicionado for menor que 3 ou se o valor subtraído for no mínimo 0.
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j]
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:
                continue
            frg.append(novoNo)

A busca em profundidade é uma técnica de busca que explora o caminho mais profundo possível a partir do nó inicial antes de retroceder. Funciona como uma pilha, onde os nós são adicionados ao início da lista e removidos do início da lista.

Tem a vantagem de usar menos memória do que a busca em largura, pois não precisa armazenar todos os nós no nível atual. Além disso, pode ser mais eficiente em encontrar soluções profundas em grafos grandes ou quando a solução está mais próxima das folhas. Por outro lado, pode não encontrar uma solução mesmo que uma exista, especialmente se o grafo tiver ciclos, pois pode entrar em loops infinitos. Além disso, não garante encontrar a solução mais curta; pode encontrar uma solução que não seja a ideal em termos de número de movimentos. Nessa abordagem "original" onde não tem mecanismos para detectar nós já visitados, ela pode explorar repetidamente os mesmos nós, e isso aumenta o tempo de execução.

In [7]:
def BuscaProfundidade(estadoInicial, estadoParada):
    no = No(estadoInicial)
    frg = [no]
    cnt = 0
    while True:
        cnt += 1

        if frg == []:
            return False
        no = frg.pop(0)
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(cnt))
            return no.caminho
        x, y = no.pos
        #Mesma lógica
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j] #troca
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:
                continue
            frg.insert(0, novoNo)

A busca por profundidade limitada (DLS) é uma técnica de busca em grafos que explora até uma certa profundidade antes de retroceder, prevenindo a exploração excessiva de caminhos profundos.

A principal vantagem da busca por profundidade limitada é o controle de profundidade, que evita a exploração excessiva de caminhos, o que é particularmente útil em grafos grandes ou com ciclos. Além disso, ela consome menos memória em comparação com a busca em largura, pois não precisa armazenar todos os nós no nível atual.

Entretanto, a busca por profundidade limitada tem desvantagens. Ela não é completa, pois pode não encontrar uma solução se o limite de profundidade for insuficiente. Também não é ótima, já que pode não encontrar a solução mais curta devido à limitação da profundidade de exploração.

Em relação à busca por profundidade normal, a busca por profundidade limitada difere principalmente pelo controle de profundidade. Enquanto a busca por profundidade normal pode explorar caminhos indefinidamente, potencialmente entrando em loops infinitos em grafos com ciclos, a busca por profundidade limitada define um limite para evitar esse problema. Isso torna a busca por profundidade limitada mais segura em termos de evitar loops, mas pode levar à omissão de soluções que estão além do limite definido.


In [8]:
def BuscaProfundidadeLimitada(estadoInicial, estadoParada, limite):
    def DLS(no, estadoParada, limite):
        if VerificaPosicao(no, estadoParada):
            return no.caminho
        elif limite == 0:
            return None

        x, y = no.pos
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i + x < 0 or i + x >= 3 or j + y < 0 or j + y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x + i][y + j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x + i][y + j]  # troca
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:
                continue
            resultado = DLS(novoNo, estadoParada, limite - 1)
            if resultado is not None:
                return resultado
        return None

    no = No(estadoInicial)
    return DLS(no, estadoParada, limite)

A busca por aprofundamento iterativo é uma técnica de busca que combina a profundidade limitada e a iteração, incrementando gradualmente o limite de profundidade até encontrar a solução.

Uma das principais vantagens da busca por aprofundamento iterativo é que ela é completa, sempre encontrando uma solução se uma solução existir. Além disso, ela é ótima, encontrando a solução mais curta em termos de número de movimentos. Outro benefício é o menor consumo de memória em comparação com a busca em largura, pois cada iteração é uma busca em profundidade limitada, que usa menos memória.

No entanto, a busca por aprofundamento iterativo tem algumas desvantagens. Pode ser mais lenta do que outras buscas, pois reexplora nós em cada iteração. Isso resulta na repetição de exploração, onde nós são explorados múltiplas vezes, aumentando o tempo total de execução.

Em comparação com a busca por profundidade normal e a busca por profundidade limitada, a busca por aprofundamento iterativo tem diferenças significativas. A busca por profundidade normal pode entrar em loops infinitos em grafos com ciclos e não garante encontrar a solução mais curta. A busca por profundidade limitada evita a exploração excessiva de caminhos profundos, mas pode não encontrar uma solução se o limite de profundidade for insuficiente e não garante a solução mais curta.

Por outro lado, a busca por aprofundamento iterativo combina o melhor de ambos os métodos. Ela evita os loops infinitos e a limitação de profundidade inadequada ao incrementar gradualmente o limite, garantindo tanto a completude quanto a otimalidade. Embora a reexploração de nós em cada iteração possa aumentar o tempo total de execução, ela assegura que todas as possíveis profundidades sejam exploradas de maneira controlada, encontrando a solução mais eficiente em termos de movimentos.

In [9]:
def BuscaAprofundamentoIterativo(estadoInicial, estadoParada):
    profundidade = 0
    while True:
        resultado = BuscaProfundidadeLimitada(estadoInicial, estadoParada, profundidade)
        if resultado is not None:
            print("Solução encontrada com profundidade limite de {}".format(profundidade))
            return resultado
        profundidade += 1

In [23]:
def AEstrela(estadoInicial, estadoParada, heuristica):
    tipoHeuristica = ["Hamming", "Manhattan"]
    if heuristica not in tipoHeuristica:
        print("Opção inválida")
    elif heuristica == "Hamming":
        hn = lambda no: sum([sum([1 for i in range(3) for j in range(3) if no.estado.array[i][j] != estadoParada.array[i][j]])])
    elif heuristica == "Manhattan":
        hn = lambda no: sum([sum([abs(i-j) for i, j in zip(*no.estado.array.nonzero())])])



    no = No(estadoInicial)
    frg = FilaPrioridade()
    frg.put(no, 0)
    cnt = 0
    while True:
        cnt += 1

        if frg.EspacoVazio():
            return False
        no = frg.get()
        if VerificaPosicao(no, estadoParada):
            print("Solução encontrada em {} movimentos".format(len(no.caminho) -1) + " utilizando heurística de " + heuristica)
            return no.caminho
        x, y = no.pos
        for i, j in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if i+x < 0 or i+x >= 3 or j+y < 0 or j+y >= 3:
                continue
            novoEstado = deepcopy(no.estado)
            novoEstado.array[x+i][y+j], novoEstado.array[x][y] = novoEstado.array[x][y], novoEstado.array[x+i][y+j]
            novoNo = no.AlcancaNovoEstado(novoEstado)
            if novoNo.estado in no.caminho:
                continue
            frg.put(novoNo, len(novoNo.caminho)-1 + hn(novoNo))

A busca A* Estrela se destaca como uma técnica de busca por sua capacidade de encontrar o caminho mais curto para o objetivo, desde que a heurística utilizada seja admissível, ou seja, nunca superestime o custo real. Essa propriedade garante que o A* sempre encontre uma solução, se existir, e que essa solução seja a mais eficiente em termos de custo.

Além disso, a flexibilidade do A* é notável, já que pode ser adaptado para uma variedade de problemas e heurísticas. Isso significa que ele pode ser aplicado em diversos cenários, desde que uma heurística apropriada seja definida.

Por outro lado, a busca A* consome uma quantidade significativa de memória, especialmente para problemas grandes, pois precisa manter todos os caminhos possíveis na memória. Essa característica pode limitar sua aplicabilidade em problemas com restrições severas de memória ou em ambientes com recursos limitados.

O desempenho da busca A* também está fortemente ligado à qualidade da heurística utilizada. Heurísticas inadequadas podem levar a um desempenho abaixo do ideal, prejudicando a eficiência da busca.

Comparado com outras técnicas de busca, como busca em largura, busca em profundidade, busca por profundidade limitada e busca por aprofundamento iterativo, o A* se destaca por sua abordagem mais direcionada, priorizando nós com menor custo total estimado. Enquanto a busca em largura explora todos os nós em um determinado nível antes de passar para o próximo, o A* avalia os nós com base em uma combinação de custo atual e estimativa do custo futuro.

Diferentemente da busca em profundidade, que busca o caminho mais profundo possível antes de retroceder, o A* considera tanto o custo atual quanto uma estimativa do custo futuro para decidir qual nó explorar em seguida. Em contraste com a busca por profundidade limitada, que define um limite fixo para a profundidade de exploração, o A* não possui tal limite, podendo adaptar dinamicamente a profundidade de exploração com base nos custos estimados dos nós. Por fim, em comparação com a busca por aprofundamento iterativo, o A* não requer iterações adicionais para ajustar o limite de profundidade, pois avalia os nós com base na função de custo e na heurística em cada passo da busca.

In [11]:
solucao = BuscaAprofundamentoIterativo(posicaoInicial, posicaoFinal)
for passo in solucao:
    print(passo)

Solução encontrada com profundidade limite de 14
[5 2 6]
[1 0 3]
[4 7 8]

[5 2 6]
[1 3 0]
[4 7 8]

[5 2 0]
[1 3 6]
[4 7 8]

[5 0 2]
[1 3 6]
[4 7 8]

[0 5 2]
[1 3 6]
[4 7 8]

[1 5 2]
[0 3 6]
[4 7 8]

[1 5 2]
[4 3 6]
[0 7 8]

[1 5 2]
[4 3 6]
[7 0 8]

[1 5 2]
[4 3 6]
[7 8 0]

[1 5 2]
[4 3 0]
[7 8 6]

[1 5 2]
[4 0 3]
[7 8 6]

[1 0 2]
[4 5 3]
[7 8 6]

[1 2 0]
[4 5 3]
[7 8 6]

[1 2 3]
[4 5 0]
[7 8 6]

[1 2 3]
[4 5 6]
[7 8 0]



In [22]:
BuscaLargura(posicaoInicial, posicaoFinal)

Solução encontrada em 14 movimentos


[[5 2 6]
 [1 0 3]
 [4 7 8],
 [5 2 6]
 [1 3 0]
 [4 7 8],
 [5 2 0]
 [1 3 6]
 [4 7 8],
 [5 0 2]
 [1 3 6]
 [4 7 8],
 [0 5 2]
 [1 3 6]
 [4 7 8],
 [1 5 2]
 [0 3 6]
 [4 7 8],
 [1 5 2]
 [4 3 6]
 [0 7 8],
 [1 5 2]
 [4 3 6]
 [7 0 8],
 [1 5 2]
 [4 3 6]
 [7 8 0],
 [1 5 2]
 [4 3 0]
 [7 8 6],
 [1 5 2]
 [4 0 3]
 [7 8 6],
 [1 0 2]
 [4 5 3]
 [7 8 6],
 [1 2 0]
 [4 5 3]
 [7 8 6],
 [1 2 3]
 [4 5 0]
 [7 8 6],
 [1 2 3]
 [4 5 6]
 [7 8 0]]

A priori, para a posição inicial que eu tinha definido anteriormente, a busca em profundidade entra em loop, considerando isso, entrei com um outro estado inicial.

In [30]:
#solucaoProfundidade = BuscaProfundidade(posicaoInicial, posicaoFinal)
solucaoProfundidade = BuscaProfundidade(Estado([[1, 2, 3], [5, 0, 6], [4, 7, 8]]), posicaoFinal)
for passo in solucaoProfundidade:
    print(passo)

Solução encontrada em 27 movimentos
[1 2 3]
[5 0 6]
[4 7 8]

[1 2 3]
[5 6 0]
[4 7 8]

[1 2 3]
[5 6 8]
[4 7 0]

[1 2 3]
[5 6 8]
[4 0 7]

[1 2 3]
[5 6 8]
[0 4 7]

[1 2 3]
[0 6 8]
[5 4 7]

[1 2 3]
[6 0 8]
[5 4 7]

[1 2 3]
[6 8 0]
[5 4 7]

[1 2 3]
[6 8 7]
[5 4 0]

[1 2 3]
[6 8 7]
[5 0 4]

[1 2 3]
[6 8 7]
[0 5 4]

[1 2 3]
[0 8 7]
[6 5 4]

[1 2 3]
[8 0 7]
[6 5 4]

[1 2 3]
[8 7 0]
[6 5 4]

[1 2 3]
[8 7 4]
[6 5 0]

[1 2 3]
[8 7 4]
[6 0 5]

[1 2 3]
[8 7 4]
[0 6 5]

[1 2 3]
[0 7 4]
[8 6 5]

[1 2 3]
[7 0 4]
[8 6 5]

[1 2 3]
[7 4 0]
[8 6 5]

[1 2 3]
[7 4 5]
[8 6 0]

[1 2 3]
[7 4 5]
[8 0 6]

[1 2 3]
[7 4 5]
[0 8 6]

[1 2 3]
[0 4 5]
[7 8 6]

[1 2 3]
[4 0 5]
[7 8 6]

[1 2 3]
[4 5 0]
[7 8 6]

[1 2 3]
[4 5 6]
[7 8 0]



In [29]:
tipoHeuristica = ["Hamming", "Manhattan"]
solucaoHamming = AEstrela(posicaoInicial, posicaoFinal, tipoHeuristica[0])
for passo in solucaoHamming:
    print(passo)
print("\n" + "="*80 + "\n")
solucaoManhattan = AEstrela(posicaoInicial, posicaoFinal, tipoHeuristica[1])
for passo in solucaoManhattan:
    print(passo)

Solução encontrada em 14 movimentos utilizando heurística de Hamming
[5 2 6]
[1 0 3]
[4 7 8]

[5 2 6]
[1 3 0]
[4 7 8]

[5 2 0]
[1 3 6]
[4 7 8]

[5 0 2]
[1 3 6]
[4 7 8]

[0 5 2]
[1 3 6]
[4 7 8]

[1 5 2]
[0 3 6]
[4 7 8]

[1 5 2]
[4 3 6]
[0 7 8]

[1 5 2]
[4 3 6]
[7 0 8]

[1 5 2]
[4 3 6]
[7 8 0]

[1 5 2]
[4 3 0]
[7 8 6]

[1 5 2]
[4 0 3]
[7 8 6]

[1 0 2]
[4 5 3]
[7 8 6]

[1 2 0]
[4 5 3]
[7 8 6]

[1 2 3]
[4 5 0]
[7 8 6]

[1 2 3]
[4 5 6]
[7 8 0]



Solução encontrada em 14 movimentos utilizando heurística de Manhattan
[5 2 6]
[1 0 3]
[4 7 8]

[5 2 6]
[1 3 0]
[4 7 8]

[5 2 0]
[1 3 6]
[4 7 8]

[5 0 2]
[1 3 6]
[4 7 8]

[0 5 2]
[1 3 6]
[4 7 8]

[1 5 2]
[0 3 6]
[4 7 8]

[1 5 2]
[4 3 6]
[0 7 8]

[1 5 2]
[4 3 6]
[7 0 8]

[1 5 2]
[4 3 6]
[7 8 0]

[1 5 2]
[4 3 0]
[7 8 6]

[1 5 2]
[4 0 3]
[7 8 6]

[1 0 2]
[4 5 3]
[7 8 6]

[1 2 0]
[4 5 3]
[7 8 6]

[1 2 3]
[4 5 0]
[7 8 6]

[1 2 3]
[4 5 6]
[7 8 0]

